In [22]:
import numpy as np
import pandas as pd
import xgboost as xgb
from config import *
from cal_accuracy import *
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [2]:
# 加载模型并使用
loaded_model = xgb.Booster()
loaded_model.load_model("xgboost_model_test.model")

In [3]:
test_data_path = r'./input/test_features_100_columns.xlsx'
test_data = pd.read_excel(test_data_path)
X_test = test_data[used_feature_columns].values
y_test = test_data['score']
dtest = xgb.DMatrix(X_test)
y_pred = loaded_model.predict(dtest)

In [28]:
contrast = pd.DataFrame()
contrast['true'] = y_test
contrast['predict'] = y_pred
contrast['difference'] = abs(y_pred - y_test)
contrast['file_name'] = test_data['file_name']
contrast

,true,predict,difference,file_name
0,4.575,4.395761,0.179239,ba1-M301-1731.wav
1,5.100,5.250212,0.150212,ba2-M302-0823.wav
2,5.150,5.866136,0.716136,ba3-M303-0743.wav
3,3.150,4.880989,1.730989,ba4-M304-1030.wav
4,5.125,4.751622,0.373378,ba5-M306-0708.wav
...,...,...,...,...
75,5.925,4.430910,1.494090,ns11-N601-1050.wav
76,5.200,4.469306,0.730694,ns12-N602-0859.wav
77,4.250,4.681808,0.431808,ns13-N603-1527.wav
78,5.750,4.705876,1.044124,ns14-N604-1029.wav


In [9]:
threshold = 1
count = (contrast['difference'] < threshold).sum()
acc = count / len(contrast)
acc

0.6375

In [12]:
avg_diff = contrast['difference'].sum() / len(contrast)
# avg_percent = avg_diff / 8
avg_percent = avg_diff / 5
(1 - avg_percent)

0.8433813374042511

In [18]:
r2 = r2_score(contrast['true'], contrast['predict'])
r2

0.14743619426097476

In [20]:
predict_class_list = []
true_class_list = []
for i in range(len(contrast)):
    predict_class, true_class = classify(contrast.predict[i], contrast.true[i])
    predict_class_list.append(predict_class)
    true_class_list.append(true_class)

acc = accuracy(predict_class_list, true_class_list)
acc

0.7875

In [25]:
mse = mean_squared_error(contrast['predict'], contrast['true'])
rmse = np.sqrt(mse)
mae = mean_absolute_error(contrast['predict'], contrast['true'])
mse, rmse, mae

(0.9696680236574089, 0.9847172303039127, 0.7830933129787445)